In [ ]:
#r "nuget: Microsoft.Extensions.Configuration"
#r "nuget: Microsoft.Extensions.Configuration.Json"
#r "nuget: Neo4j.Driver"

using Neo4j.Driver;
using System.Text.Json;


In [ ]:
#!import config/Config.cs

In [ ]:
Console.WriteLine($"Neo4j Uri {Config.Neo4jUri}");
Console.WriteLine($"Neo4j User {Config.Neo4jUser}");
Console.WriteLine($"Neo4j Password {Config.Neo4jPassword}");

In [ ]:
var driver = GraphDatabase.Driver(Config.Neo4jUri, AuthTokens.Basic(Config.Neo4jUser, Config.Neo4jPassword));
await driver.VerifyConnectivityAsync();

## Working with Cypher Results

### hello world!

The simplest Cypher query is a `RETURN` clause that returns a string literal. This query returns the string "Hello, World!".

Notice that the `Result` below contains two parts:

1. `Result` showing the data returned by the query.
2. `Summary` showing information about the query execution.

In [ ]:
driver.ExecutableQuery("RETURN 'hello world' as message").ExecuteAsync().Result

### Get specific result value

The `Result.Result` is a list of IRecord objects. Each IRecord object contains a list of key-value pairs. You can access the value of a specific key by using either:

1. `<T>Get(string key)` to cast the value to a specific type.
2. `<T>As<T>(string key)` to cast the value to a specific type.

In [ ]:
var gotMessage = driver.ExecutableQuery("RETURN 'hello world' as message").ExecuteAsync().Result.Result[0].Get<string>("message");
var gotField = driver.ExecutableQuery("RETURN 'hello world' as message").ExecuteAsync().Result.Result[0]["message"];
var gotFieldAs = driver.ExecutableQuery("RETURN 'hello world' as message").ExecuteAsync().Result.Result[0]["message"].As<string>();

Console.WriteLine($"Got message: {gotMessage}");
Console.WriteLine($"Got field: {gotField}");
Console.WriteLine($"Got field as: {gotFieldAs}");

### map each result row

Add `WithMap()` to apply a map function to each result row. 

Here, we're converting each row from a record to a simple string...

In [ ]:
driver.ExecutableQuery("RETURN 'hello world' as message")
  .WithMap(row => row["message"].As<string>())
  .ExecuteAsync().Result.Result[0]

## Create data

### Create a Node

As graph query language, Cypher works with patterns of data. Patterns are used to create and find data.

The simplest pattern is a single "node", which is what data records are called in a graph database.

The following query creates a single node, assigning the variable `a` to that node and returning the `a` node.

In [ ]:
driver.ExecutableQuery("CREATE (a) RETURN a").ExecuteAsync().Result.Result

### Create a Node with a Label

Nodes can belong to one or more sets identified by "label". You can set a label when creating a node using the
notation `(:NameOfLabel)`.

For example, the following query creates a node with the label `Person`.

In [ ]:
driver.ExecutableQuery("CREATE (a:Person) RETURN a").ExecuteAsync().Result.Result

### Create a Node with Properties

As data records, nodes can also carry properties. Properties are key-value pairs.

The following query creates a node with the label `Person` and two properties: `name` and `height`.

In [ ]:
driver.ExecutableQuery("CREATE (a:Person {name:'Andreas', height: 186}) RETURN a").ExecuteAsync().Result.Result

## Find data

Finding or reading data also uses patterns. You can use similar single-node patterns.

### Find all Nodes

The following query finds all nodes in the graph by using the `MATCH` clause with a single node pattern.

Like `CREATE`, the `MATCH` clause binds variables within a pattern which can be used for selecting results.

Here, the `all` variable will match any node within the graph, producing as many result rows as there are nodes. For larger graphs, this is probably a bad idea.

In [ ]:
driver.ExecutableQuery("MATCH (all) RETURN all").ExecuteAsync().Result.Result

### Find Nodes with a Label

You can filter nodes by label by adding the label to the node pattern.

In [ ]:
driver.ExecutableQuery("MATCH (p:Person) RETURN p").ExecuteAsync().Result.Result

### Find Nodes with Properties

You can filter nodes by properties by adding the property to the node pattern.

In [ ]:
driver.ExecutableQuery("MATCH (andreas:Person {name:'Andreas'}) RETURN andreas").ExecuteAsync().Result.Result

### Find Nodes using a `WHERE` clause

The `WHERE` clause allows you to filter results based on conditions.

In [ ]:
driver.ExecutableQuery("MATCH (a:Person) WHERE a.name STARTS WITH 'A' RETURN a").ExecuteAsync().Result.Result

## Upserting data

If you re-run the `CREATE` statements from above, you'll create potentially duplicate data. To avoid this, you can use the `MERGE` clause, which acts like an "upsert" (update or insert) operation.

`MERGE` happens in two phases:

1. `MATCH` phase: The query tries to find a node that matches the pattern
2. `CREATE` phase: If no node is found, a new node is created that matches the pattern

The matching works with an exact match of the criteria in the pattern. 

In [ ]:
driver.ExecutableQuery("MERGE (a:Person {name:'Andreas'}) RETURN a").ExecuteAsync().Result.Result

### Set values on `MERGE`

You can use the `SET` clause to set properties on the node found or creating by `MERGE`.  

In [ ]:
driver.ExecutableQuery("""
  MERGE (a:Person {name:'Andreas'}) 
  SET a.height = 190
  RETURN a
  """
).ExecuteAsync().Result.Result

### Set values on `MERGE` with `ON CREATE`

`MERGE` has a special sub-clause that only exectutes when the `CREATE` phase is triggered. This is the `ON CREATE` clause.

To conditionally set properties only when a new node is created, use the `ON CREATE` clause.

This example uses the built-in `datetime()` function to set the `createdAt` property only when the node is created.

In [ ]:
driver.ExecutableQuery("""
  MERGE (a:Person {name:'Nigel'}) 
  ON CREATE SET a.createdAt = datetime()
  RETURN a
  """
).ExecuteAsync().Result.Result

## Set values on `MERGE` with `ON MATCH`

Similarly, `MERGE` also has an `ON MATCH` sub-clause that only executes when the `MATCH` phase is triggered.

In [ ]:
driver.ExecutableQuery("""
  MERGE (a:Person {name:'Nigel'}) 
  ON MATCH SET a.lastRead = datetime()
  RETURN a
  """
).ExecuteAsync().Result.Result